# 09 - 종합 실습

## (1) 환경 준비

### 1) 라이브러리 불러오기

In [1]:
# 판다스, 넘파이 라이브러리 불러오기
import pandas as pd
import numpy as np

### 2) 데이터 불러오기

다음 세개의 데이터를 불러와서 저장합시다.

* sales : https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv
* products : https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv
* customers : https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv

In [2]:
sales = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/sales.csv')
products = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/products.csv')
customers = pd.read_csv('https://raw.githubusercontent.com/DA4BAM/dataset/master/customers2.csv')

세 데이터프레임의 상위 5개 행을 조회하시오.

In [3]:
sales.head(5)

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID
0,107,2,2016-01-02,p1036481,2,2100,c150417
1,69,1,2016-01-02,p1152861,1,1091,c212716
2,69,7,2016-01-02,p1013161,1,2600,c212716
3,69,8,2016-01-02,p1005771,1,1650,c212716
4,69,11,2016-01-02,p1089531,1,2600,c212716


In [4]:
products.head(5)

,ProductID,ProductName,Category,SubCategory,CategoryOrd
0,p1052661,새우깡,간식,과자,3
1,p1054261,고구마스틱,간식,과자,3
2,p1097821,짱구,간식,과자,3
3,p1097831,감자칩,간식,과자,3
4,p1119071,뿌셔뿌셔,간식,과자,3


In [5]:
customers.head(5)

,CustomerID,RegisterDate,Gender,BirthYear
0,c328222,2014-09-25,F,1960
1,c281448,2013-06-18,F,1974
2,c038336,2003-10-10,F,1968
3,c084237,2007-03-09,F,1982
4,c162600,2010-06-14,F,1978


## (2) 데이터 집계, 수정, 조회하기

### 1) sales의 ProductID 별 판매량(Qty)과 판매액(Amt)의 합계를 tmp에 저장하고 조회하시오.

In [6]:
tmp = sales[['Qty', 'Amt']].groupby(sales['ProductID']).sum()
tmp

,Qty,Amt
ProductID,,
p1001771,1055,3354827
p1002841,903,11011541
p1005621,906,2601703
p1005771,3963,7319963
p1005891,5194,10119037
...,...,...
p1246581,2180,2456346
p1255281,809,955633
p1256521,701,1241545


### 2)  1번 결과를 판매액 기준 내림차순으로 정렬하고 상위 5개 상품을 조회하시오.  
(매출액 상위 5개 상품을 조회하시오.)

In [7]:
tmp.sort_values('Amt', ascending=False).head(5)

,Qty,Amt
ProductID,,
p1072601,4058,18129067
p1178011,1653,14078818
p1002841,903,11011541
p1005891,5194,10119037
p1194801,990,7517664


### 3) customers의 성별(Gender) 별 고객 수를 조회하시오.

In [53]:
customers.groupby('Gender').size()


Gender
F    2060
M     183
dtype: int64

### 4) customers의 출생연도(BirthYear) 로 부터 나이(Age)를 계산해서 변수로 추가하시오.

In [23]:
import datetime as dt
today = dt.date.today().year

In [25]:
customers['Age'] = today - customers['BirthYear']

In [26]:
customers.head(5)

,CustomerID,RegisterDate,Gender,BirthYear,Age
0,c328222,2014-09-25,F,1960,62
1,c281448,2013-06-18,F,1974,48
2,c038336,2003-10-10,F,1968,54
3,c084237,2007-03-09,F,1982,40
4,c162600,2010-06-14,F,1978,44


### 5) customers의 나이(Age)를 나이대(AgeGroup)로 변환하시오.

* [ , 30)  : 30미만
* [30, 40) : 30대
* [40, 50) : 40대
* [50, 60) : 50대
* [60, 70) : 60대
* [70, )   : 70이상

> [30, 40) 의 의미 : 30 <= ,  < 40

In [45]:
bin = [0]
for i in range(30, 80, 10):
  bin.append(i)
bin.append(float('inf'))
customers['AgeGroup'] = pd.cut(customers['Age'], bins=bin, right=True)

In [46]:
customers.head()

,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup
0,c328222,2014-09-25,F,1960,62,"(60.0, 70.0]"
1,c281448,2013-06-18,F,1974,48,"(40.0, 50.0]"
2,c038336,2003-10-10,F,1968,54,"(50.0, 60.0]"
3,c084237,2007-03-09,F,1982,40,"(30.0, 40.0]"
4,c162600,2010-06-14,F,1978,44,"(40.0, 50.0]"


### 6) customers의 나이대(AgeGroup) 별 고객수를 구해 봅시다.

In [60]:
customers.groupby('AgeGroup').size()

AgeGroup
(0.0, 30.0]       9
(30.0, 40.0]    236
(40.0, 50.0]    966
(50.0, 60.0]    696
(60.0, 70.0]    242
(70.0, inf]      94
dtype: int64

### 7) 매출액 top 5 상품명을 조회해 봅시다.

* 매출액은 sales에 있고, 상품명은 products에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.
    * 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.
    * 2) 상품명 별 매출액 합계를 집계합니다. 
    * 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.


In [75]:
# 1) sales와 products를 ProductID 기준으로 inner 조인을 합시다.

sales.merge(products, on='ProductID').groupby('ProductID').sum().sort_values('Amt', ascending=False).head()['Amt']

ProductID
p1072601    18129067
p1178011    14078818
p1002841    11011541
p1005891    10119037
p1194801     7517664
Name: Amt, dtype: int64

In [ ]:
# 2) 2) 상품명 별 매출액 합계를 집계합니다.



In [ ]:
# 3) 매출액 합계 기준 내림차순으로 정렬하고 top 5를 조죄합니다.



### 8) 연령대 별 매출액을 조회합시다.

* 매출액은 sales에 있고, 연령대(AgeGroup)은 customers에 있습니다. 한꺼번에 조회하려면 merge를 사용해야 합니다.


In [70]:
# 1) sales와 customers를  CustomerID 기준으로 inner 조인을 합시다.

sales.merge(customers, on='CustomerID')

,OrderID,Seq,OrderDate,ProductID,Qty,Amt,CustomerID,RegisterDate,Gender,BirthYear,Age,AgeGroup
0,107,2,2016-01-02,p1036481,2,2100,c150417,2010-03-03,F,1974,48,"(40.0, 50.0]"
1,107,1,2016-01-02,p1175481,1,1300,c150417,2010-03-03,F,1974,48,"(40.0, 50.0]"
2,185,1,2016-01-04,p1162631,1,4600,c150417,2010-03-03,F,1974,48,"(40.0, 50.0]"
3,67,2,2016-01-11,p1012751,1,1350,c150417,2010-03-03,F,1974,48,"(40.0, 50.0]"
4,201,3,2016-01-12,p1005891,1,1950,c150417,2010-03-03,F,1974,48,"(40.0, 50.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
70007,193,5,2017-03-31,p1179241,2,4400,c017735,2002-07-31,F,1960,62,"(60.0, 70.0]"
70008,193,6,2017-03-31,p1144661,1,2500,c017735,2002-07-31,F,1960,62,"(60.0, 70.0]"
70009,193,7,2017-03-31,p1012581,1,1050,c017735,2002-07-31,F,1960,62,"(60.0, 70.0]"
70010,193,9,2017-03-31,p1052661,1,1545,c017735,2002-07-31,F,1960,62,"(60.0, 70.0]"


In [74]:
# 2) AgeGroup 별 매출액 합계를 집계합니다.

sales.merge(customers, on='CustomerID').groupby('AgeGroup').sum()['Amt']

AgeGroup
(0.0, 30.0]       283767
(30.0, 40.0]    15894418
(40.0, 50.0]    90520765
(50.0, 60.0]    63994308
(60.0, 70.0]    18295892
(70.0, inf]      6010318
Name: Amt, dtype: int64

### 9) [심화]연령대 별, 상품 카테고리 매출 비중(%)을 조회합시다.

* 1) 세 데이터프레임을 merge 해야 합니다.
    * merge는 한번에 두개씩만 가능하므로, 두번 수행합니다.
* 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다. 
    * 하나의 연령대 값에 대해서, 
        * 상품 카테고리(Category)별 매출액 합계를 구합니다.
        * 연령대에 해당하는 전체 매출액으로 나눠서, 카테고리별 비율을 계산합니다.
        * 결과를 출력합니다.



In [ ]:
# 1) 세 데이터프레임을 merge 



In [ ]:
# 2) for loop를 이용하여, 각 연령대에 대해서 다음의 작업을 반복 수행합니다

